In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

dataset = load_dataset("json", data_files={"train": "FinalData/train.jsonl", "validation": "FinalData/val.jsonl"})

def preprocess_function(batch):
    inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=256)
    targets = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=256)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized = dataset.map(preprocess_function, batched=True)


Map: 100%|██████████| 3730/3730 [00:00<00:00, 4867.49 examples/s]


In [1]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./flan_t5_mathqa_finetuned",
    eval_strategy="epoch", 
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    fp16=True,
    save_total_limit=2,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)
print("✅ TrainingArguments OK")


c:\Projects\P1_NLP\FormulaVerse\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: Could not import module 'get_available_reporting_integrations'. Are this object's requirements defined correctly?